In [2]:
import pandas as pd

In [9]:
# Define the path to the Excel file
file_path = "./data/Dataset.xlsx"
# Read the Excel file into a pandas DataFrame
car_insur_df = pd.read_excel(file_path, sheet_name="Sheet1")
# Print the shape of the DataFrame
print(f"Shape of the DataFrame: {car_insur_df.shape}")
# Print the column names of the DataFrame
print(f"Columns of the DataFrame: {list(car_insur_df.columns)}")
# Display the first five rows of the DataFrame
car_insur_df.head()

Shape of the DataFrame: (11565, 34)
Columns of the DataFrame: ['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex', 'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory', 'VehiclePrice', 'FraudFound_P', 'PolicyNumber', 'RepNumber', 'Deductible', 'DriverRating', 'Days_Policy_Accident', 'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType', 'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year', 'BasePolicy', 'ClaimSize']


,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,55526.069421
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,59294.455571
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,71756.941320
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability,7584.153493
4,Feb,1,Saturday,Honda,Urban,Monday,Feb,3,Male,Married,...,36 to 40,No,No,External,1 to 2,no change,1 vehicle,1994,Collision,98545.374339


# Data Cleaning
## Remove rows with missing values


In [10]:
print(car_insur_df.shape)
car_insur_df = car_insur_df.dropna()
print(car_insur_df.shape)

(11565, 34)
(11554, 34)


## Remove column "Age" due to invalid zero values

In [11]:
car_insur_df[car_insur_df.Age != 0]["AgeOfPolicyHolder"].value_counts()

AgeOfPolicyHolder
31 to 35    4191
36 to 40    3005
41 to 50    2119
51 to 65    1067
26 to 30     445
over 65      388
21 to 25      73
18 to 20      13
Name: count, dtype: int64

In [12]:
del car_insur_df["Age"]

## Remove row with invalid value 0 for MonthClaimed

In [13]:
car_insur_df = car_insur_df[car_insur_df.MonthClaimed != 0]

## Remove redundant columns

In [14]:
# PolicyType is a combination of VehicleType and BasePolicy
del car_insur_df["PolicyType"]

## Get Date for Claims and Accidents

In [15]:
from datetime import datetime, timedelta


def get_date(year, month, week_of_month, day_of_week):
    # Create a dictionary to map the month and day of the week strings to their corresponding integers
    date_dict = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
        "Monday": 0,
        "Tuesday": 1,
        "Wednesday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6,
    }

    # Check if the month and day of the week strings are valid
    if month not in date_dict or day_of_week not in date_dict:
        raise ValueError("Invalid month or day of the week")

    # Calculate the starting date of the month
    first_day_of_month = datetime(year, date_dict[month], 1)

    # Find the first occurrence of the given day of the week in the month
    current_date = first_day_of_month
    while current_date.weekday() != date_dict[day_of_week]:
        current_date += timedelta(days=1)

    # Calculate the target date by adding the week offset
    target_date = current_date + timedelta(weeks=week_of_month - 1)

    # Check if the target date is in the same month as the first occurrence of the day of the week
    if target_date.month != current_date.month:
        target_date -= timedelta(weeks=1)

    return target_date


def calculate_accident_dates(df):
    accident_dates = []
    for _, row in df.iterrows():
        year, month, week_of_month, day_of_week = (
            row["Year"],
            row["Month"],
            row["WeekOfMonth"],
            row["DayOfWeek"],
        )
        accident_date = get_date(year, month, week_of_month, day_of_week)
        accident_dates.append(accident_date)
    df["AccidentDate"] = accident_dates
    return df


def calculate_claim_dates(df):
    claim_dates = []
    for _, row in df.iterrows():
        month_claimed, week_of_month_claimed, day_of_week_claimed = (
            row["MonthClaimed"],
            row["WeekOfMonthClaimed"],
            row["DayOfWeekClaimed"],
        )
        accident_date = row["AccidentDate"]
        # If the month claimed is before the accident month, add one year to the accident year
        if accident_date.month > datetime.strptime(month_claimed, "%b").month:
            year_claimed = accident_date.year + 1
        else:
            year_claimed = accident_date.year
        claim_date = get_date(
            year_claimed, month_claimed, week_of_month_claimed, day_of_week_claimed
        )
        claim_dates.append(claim_date)
    df["ClaimDate"] = claim_dates
    return df


def calculate_days_between_dates(df):
    df["DaysAccidentClaim"] = (df["ClaimDate"] - df["AccidentDate"]).dt.days
    return df


def calculate_dates(df):
    df = calculate_accident_dates(df)
    df = calculate_claim_dates(df)
    df = calculate_days_between_dates(df)
    return df


car_insur_df = calculate_dates(car_insur_df)

In [16]:
car_insur_df.DaysAccidentClaim.value_counts()

DaysAccidentClaim
1      1516
0      1499
2       870
3       683
4       545
       ... 
109       1
163       1
247       1
170       1
278       1
Name: count, Length: 209, dtype: int64

In [17]:
car_insur_df.head(3)

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize,AccidentDate,ClaimDate,DaysAccidentClaim
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,External,none,1 year,3 to 4,1994,Liability,55526.069421,1994-12-28,1995-01-03,6
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,External,none,no change,1 vehicle,1994,Collision,59294.455571,1994-01-19,1994-01-24,5
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,External,none,no change,1 vehicle,1994,Collision,71756.941320,1994-10-28,1994-11-10,13


In [19]:
car_insur_df.to_csv("./data/car_insurance_data_cleaned.csv", index=False)